In [1]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import PolyAnnotation, Range1d
from bokeh.embed import components
from bokeh.transform import jitter
import os
import pandas as pd
modelname = '1615871212'
from flask import url_for

model_folder_path = os.path.join(os.getcwd(), 'static', 'modelpredictions', modelname)
model_predictions = os.path.join(model_folder_path, 'predictions.csv')
model_strings = os.path.join(model_folder_path, 'strings.csv') #Not used currently

In [2]:
df = pd.read_csv(model_predictions)
df['y'] = .5
get_file_name = lambda row: row['ResizedPath'].split('\\')[-1]
df['FileName'] = df.apply(get_file_name, axis=1)
#create_URL = lambda row: url_for('static', filename= 'Resized/' + row['FileName'])
#df['PhotoURL'] = df.apply(create_URL, axis=1)
babyDF = df.loc[df['Value'] == 'Baby'].reset_index()
baby_likeliness = lambda row: row['LikelyBaby'] - row['LikelyEmpty']
babyDF['BabyLikeliness'] = babyDF.apply(baby_likeliness, axis=1)

nobabyDF = df.loc[df['Value'] == 'No Baby'].reset_index()
nobaby_likeliness = lambda row: -1*(row['LikelyEmpty'] - row['LikelyBaby'])
nobabyDF['NoBabyLikeliness'] = nobabyDF.apply(nobaby_likeliness, axis=1)

In [21]:
tools = 'pan, wheel_zoom, reset'
TOOLTIPS = '<img src= "@ResizedPath">'

p = figure(
        title= f'Model {modelname} Predictions',
        tooltips= TOOLTIPS,
        tools=tools,
        active_scroll= 'wheel_zoom',
        y_range=Range1d(start=-.05, end=1.05, bounds=(-2,2)),
        x_range=Range1d(start=-1.05, end=1.05, bounds=(-2,2))
    )
baby_src = ColumnDataSource(data= babyDF)
nobaby_src = ColumnDataSource(data= nobabyDF)
JITTER_RADIUS_X = .05
JITTER_RADIUS_Y = 1
DOT_SIZE = 10
DOT_ALPHA = .1
BACK_ALPHA = .03
p.circle(
    jitter('BabyLikeliness', JITTER_RADIUS_X),
    jitter('y', JITTER_RADIUS_Y),
    source= baby_src,
    size=DOT_SIZE,
    color='blue', alpha=DOT_ALPHA,
    legend_label="Photos With Baby"
)
p.circle(
    jitter('NoBabyLikeliness', JITTER_RADIUS_X),
    jitter('y', JITTER_RADIUS_Y),
    source= nobaby_src,
    size=DOT_SIZE,
    color='red', alpha=DOT_ALPHA,
    legend_label="Photos Without Baby"
)

p.sizing_mode = 'scale_width'
red_polygon = PolyAnnotation(
    fill_color="red",
    fill_alpha=BACK_ALPHA,
    xs=[0, 0, -10, -10],
    ys=[-10, 10, 10, -10],
)

p.legend.location = "top_right"
p.legend.click_policy="hide"
p.legend.label_text_font_size = '14pt'

blue_polygon = PolyAnnotation(
    fill_color="blue",
    fill_alpha=BACK_ALPHA,
    xs=[0, 0, 10, 10],
    ys=[-10, 10, 10, -10],
)
p.add_layout(red_polygon)
p.add_layout(blue_polygon)
show(p)

'97.99'

'97.99'